In [ ]:
# !pip install -U langchain
# !pip install -U langchain_community
# !pip install -U langchain_openai
# !pip install faiss-cpu
# !pip install pypdf

In [2]:
#We stant by fetching a document that loads the book - "Atomic habits"
from langchain_community.document_loaders import PyPDFLoader

pdfloader = PyPDFLoader(r"C:\Users\kurathore\Downloads\SimpleRAG\story1.pdf")
pdfpages = pdfloader.load_and_split()

from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS 
# from langchain_openai.embeddings import OpenAIEmbeddings 
from langchain_community.embeddings import OpenAIEmbeddings
import os
os.environ["OPENAI_API_KEY"] = ""

In [3]:
# Load the document using LangChain's built in extractors, formatters, Loaders, embeddings, and
mydocuments = pdfloader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(mydocuments)
# Using OpenAI default embedding model
embeddings = OpenAIEmbeddings()

# Using LangChain API functions to interact with FAISS
db = FAISS.from_documents(texts, embeddings)

c:\Users\kurathore\AppData\Local\anaconda3\envs\RAG_APP\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [14]:
#Using Langchain retreiver

retriever = db.as_retriever()
from langchain.agents.agent_toolkits import create_retriever_tool
tool = create_retriever_tool(
    retriever,
    "story1",
    "Searches and returns documents regarding the children story"
)
tools = [tool]

#Wrapping LLM with conversational agent

from langchain.agents.agent_toolkits import create_conversational_retrieval_agent

# LangChain given API to interact with chat models
# from langchain_openai.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature = 0)
agent_executor = create_conversational_retrieval_agent(llm, tools, verbose=True)

In [17]:
inputt = "Read all the stories. Take inspiration from all the stories and create a short sequel story."
result = agent_executor.invoke({"input": inputt})



> Entering new AgentExecutor chain...
Once upon a time, in a magical forest, there lived a wise owl named Oliver and a playful squirrel named Sammy. Oliver was known throughout the forest for his wisdom and guidance, while Sammy was loved by all for his cheerful and mischievous nature.

One day, a mysterious fog descended upon the forest, causing confusion and fear among the animals. Oliver, sensing the unease, called for a meeting with all the forest creatures to discuss a solution. As they gathered, Sammy, always eager for an adventure, proposed seeking the help of the ancient Tree of Wisdom, rumored to hold the key to dispelling the fog.

With Oliver's wisdom and Sammy's enthusiasm, the duo set out on a journey through the dense forest, facing challenges and obstacles along the way. They encountered talking trees, mischievous fairies, and helpful animals who guided them towards the Tree of Wisdom.

Upon reaching the ancient tree, they found it shrouded in a mystical light, emanati